In [ ]:
from src.fusion_solar_py.client import FusionSolarClient

In [ ]:
# log into the API - with proper credentials...
# Read YAML file
ROOT_DIR="."

import json
with open(f"{ROOT_DIR}/tests/credentials.json", 'r') as stream:
    cred = json.load(stream)

client = FusionSolarClient(cred['username'], cred['password'], huawei_subdomain='region04eu5')


In [ ]:
client._login()

In [ ]:
plants = client.get_plants()

In [9]:
for plant in plants:
    print(plant.name)
    # r= plant.get_plant_stats(return_resp=True)

steenhof huawei


In [ ]:
devices = client.get_devices()

In [ ]:
import json

for device in devices:
    print(device.name, device.type)
    print(json.dumps(device.get_device_stats(), indent=4))
    # r= device.get_device_stats(return_resp=True)
    # break


In [ ]:
import time
import datetime
import pandas
pandas.options.plotting.backend = "plotly"

# d = datetime.datetime.now()
# # d = datetime.datetime(2022,11,1)
# dt = datetime.datetime(
#     year=d.year,
#     month=d.month,
#     day=d.day,
# )
# query_time= int(dt.timestamp())*1000

# plant_data=client.get_plant_stats(plant_ids[0], query_time=query_time)
df=plants[0].get_plant_stats()


df.plot()

In [ ]:
df.iloc[-1].to_dict() #get latest entry

In [ ]:
url=f'https://{client._huawei_subdomain}.fusionsolar.huawei.com/rest/neteco/web/config/device/v1/device-list'
params={
    # 'conditionParams.checkShareStationDn': client._parent_id, #"NE=33559773",
    'conditionParams.parentDn':  'NE=33559773', #client._parent_id,# ,NE=34010450
    # 'conditionParams.curPage': 1,
    # 'conditionParams.recordperpage': 10,
    'conditionParams.mocTypes': '20814,20815,20816,20819,20822,50017,60066,60014,60015,23037', # specifies the types of devices
    # 'conditionParams.signals': "10001", #10046,10047,21001,21002,21003,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,50011,50012,50013,50014,50015,22049,21020,21009,50017,20004",
    '_': round(time.time() * 1000)
}
r = client._session.get(url=url, params=params)
r.raise_for_status()
r.json()

device_key={}
for device in r.json()['data']:
    device_key[device['mocTypeName']]=device['dn']
# self._device_key=device_key
device_key

In [ ]:
login_data = client._send_request(
            url=f"https://{client._huawei_subdomain}.fusionsolar.huawei.com/unisess/v1/auth/session")
client._session.headers["roarand"]=login_data['csrfToken']


url = "https://region04eu5.fusionsolar.huawei.com/rest/pvms/web/device/v1/deviceExt/set-config-signals"


data = {"dn": client._device_key['Dongle'], "changeValues": '[{"id":"230190032","value":"0"}]'} 
# 230190032 stands for "Active Power Control"
#these are the options for value:
# {0: "No limit",
# 5: "Zero Export Limitation",
# 6: "Limited Power Grid (kW)",
# 7: "Limited Power Grid (%)"}

r = client._session.post(url, data=data)

print(r.status_code)
print(r.json())